# <font color='4285f4'>G</font><font color='ea4335'>o</font><font color='fbbc05'>o</font><font color='4285f4'>g</font><font color='34a853'>l</font><font color='ea4335'>e</font>  Earth Engine 平台数据集下载方式
 <font color='red'>（全程需要科学上网）</font>  
## 利用 geemap 批量下载 Sentinel-2 数据教程 （其他数据类似）
**功能特性：**
- 可人为指定起始及终止时间
- 可根据区域矢量文件控制下载区域
- 可控制波段数
- 可控制云量(可直接去云并用多时间段的影像拼接处完整的一副图像)
- 不限制下载数量及文件大小（<font color='4285f4'>G</font><font color='ea4335'>o</font><font color='fbbc05'>o</font><font color='4285f4'>g</font><font color='34a853'>l</font><font color='ea4335'>e</font> Earth Engine 平台限制2G大小）

**本 Code 的参考教程：**
- https://mp.weixin.qq.com/s/rRCXi_c8hcM7uROjahktBw    
- https://geemap.org/notebooks/118_download_image/  

**全国矢量文件下载地址**
- [【阿里云可视化平台】](https://datav.aliyun.com/portal/school/atlas/area_selector)---导出为 json 格式


**GEE 平台的数据集目录**
- [【Google Earth Engine数据集目录】](https://developers.google.com/earth-engine/datasets/catalog/) 
- [【哨兵 2 号数据介绍】](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)   

<div align="left">
    <img src="https://github.com/AI-Tianlong/Useful-Tools/assets/50650583/61791339-c074-4e96-a0d4-7d55c9057569" alt="NVIDIA-Jetson" width="50%">  
</div>

<div align="left">
    <p></p>  
    <a href='https://github.com/AI-Tianlong'>艾天龙 AI-Tianlong</a>
    <p>2024 年 2 月 24</p>  
</div>
                          

## 第 1 步：注册 <font color='4285f4'>G</font><font color='ea4335'>o</font><font color='fbbc05'>o</font><font color='4285f4'>g</font><font color='34a853'>l</font><font color='ea4335'>e</font> Earth Engine 账号

访问 [Google Earth Engine](https://code.earthengine.google.com/) 进行注册

## 第 2 步：导入 ee 包

In [16]:
import ee  #导入ee
import geemap
from IPython.display import Image
import datetime
import os

# 使用 `pip install ATL-Tools` 安装
from ATL_Tools import mkdir_or_exist, find_data_list 


## 第 3 步：初始化项目
授权登录，这一步必须做，


In [17]:
# 身份验证，会跳转到浏览器进行验证
ee.Authenticate() 

#初始化项目，编号在GEE查看
ee.Initialize(project='applied-tractor-343704')  

## 第 4 步：加载 哨兵 2 号底图

In [3]:
sentinel2_images = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

##  第 5 步：加载 要下载区域的矢量边界
- 需要 `.shp` 文件 或 `.json` 文件  
- `.shp`文件需要该文件下包含 .shp .shx .dbf .prj 四个文件，使用 `geemap.shp_to_ee(xxx)` 函数导入  
- `.json`文件需要，使用 `geemap.geojson_to_ee(xxx)` 函数导入  



In [18]:
# shp 转换为 ee 格式
Harbin_shp = "./nangangqu/nangangqu.shp"
Harbin_ee = geemap.shp_to_ee(Harbin_shp)  

## json 转换为 ee 格式 （从最开始下载的json）
# Harbin_json = "./nangangqu/nangangqu.json"
# Harbin_ee = geemap.shp_to_ee(Harbin_json)  

## 第 6 步：哨兵 2 号去云的函数

In [5]:
# 借助 Sentinel-2 数据中的 QA60 字段去云，这是 GEE 官网提示的去云函数
def maskS2clouds(image):
   qa = image.select('QA60')
   # Bits 10 and 11是云，我们要把它mask掉
   cloudBitMask = 1 << 10
   cirrusBitMask = 1 << 1
   # 这两个标志都应该设置为0，表示条件明确。
   mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
   # 哨兵的像元值是反射率的10000倍，要除以10000
   return image.updateMask(mask).divide(10000)

## 第 7 步：筛选出待下载的影像

In [6]:
# 时间段 2022 年全年、云层覆盖率小于 10%、以目标区域为边界、遍历去云函数、选取 1~8 波段
collection = sentinel2_images.filterDate('2019-7-15', '2019-8-15') \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
            .filterBounds(Harbin_ee) \
            .map(maskS2clouds) \
            .select('B[1-8]')

## 第 8 步：裁剪图像，这里是裁切成了最小的外接矩形

In [7]:
# ROI 区域为 矢量边界最小的外接矩形
ROI = Harbin_ee.geometry().bounds()

# 将影像数据集计算中值后得到的单幅影像针对目标区域进行裁剪，得到最终待下载数据
composite = collection.median().clip(ROI)

## 第 9 步：数据显示 检查一下要下载的数据

In [8]:
rgbVis = {
 'min': 0.0,
 'max': 0.3,
 'bands': ['B4', 'B3', 'B2'],
}
# 设置显示样式：color代表边界颜色；fillcolor代表填充颜色
styling = {
   'color': 'red',
   'fillColor': '00000000'
}

**运行完下一单元代码，顺利的话，会生成如下图所示的样子**
<div align="left">
    <img src="https://github.com/AI-Tianlong/Useful-Tools/assets/50650583/34599a5b-6869-45ac-b7df-3f4db136390d" alt="NVIDIA-Jetson" width="50%">  
</div>

In [9]:
# 可视化
Map = geemap.Map()
Map.addLayer(composite, rgbVis, '南岗区S2影像')
Map.addLayer(Harbin_ee.style(**styling), {}, "南岗区边界")  # 将矢量数据添加到地图上
Map.centerObject(Harbin_ee, 10)
Map

Map(center=[45.648595197770454, 126.56683615320915], controls=(WidgetControl(options=['position', 'transparent…

## 第 10 步：可视化检查完没有问题，开始准备下载

In [10]:
# 配置输出目录
output_file = "E:\Datasets\ATL_ATL自建数据集\Sentinel2-datasets\南岗区"

mkdir_or_exist(output_file)

out_tif = os.path.join(output_file, "S2_SR_2022_nangangqu.tif")

**开始下载影像 顺利的话会出现一个进度条，如下图所示**
<div align="left">
    <img src="https://github.com/AI-Tianlong/Useful-Tools/assets/50650583/46d68958-4f1e-4e3b-84af-0c789d583064" alt="NVIDIA-Jetson" width="70%">  
</div>
一定要注意科学上网的稳定性，下载过程中断了就要重新下载，不然会出现下载不完整的情况  
最后下载的图像就会出现在指定的文件下

In [11]:
# 下载影像
geemap.download_ee_image(
   image=composite,
   filename=out_tif,
   region=ROI,
   crs="EPSG:4326",
   scale=10,
)

S2_SR_2022_nangangqu.tif: |          | 0.00/282M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None
